In [1]:
import pyspark.sql.functions as f
from pyspark.sql import Window
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()
w = Window().orderBy(f.lit('A'))

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/20 13:35:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/20 13:35:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/06/20 13:35:22 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/06/20 13:35:22 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
#Leitura de CSV
df0 = spark.read.csv('work/Recebimentos.csv',sep=';',header='false');

In [4]:
  df1 = df0.select(df0._c0.cast("string").alias('TIPO'), \
                                           df0._c1.cast("string").alias('FORMAPG'), \
                                           f.substring(df0._c2,1,2).cast("int").alias('DIA_HIS'), \
                                           f.substring(df0._c2,4,2).cast("int").alias('MES_HIS'), \
                                           f.substring(df0._c2,7,4).cast("int").alias('ANO_HIS'), \
                                           f.substring(df0._c2,12,2).cast("int").alias('HORA_HIS'), \
                                           f.substring(df0._c2,15,2).cast("int").alias('MINUTO_HIS'), \
                                            f.regexp_replace(f.regexp_replace(f.regexp_replace(f.trim(df0._c4),'[R$ ]',''),'[.]',''),',','.').alias('VALOR'),\
                                            f.regexp_replace(f.regexp_replace(f.regexp_replace(f.trim(df0._c5),'[R$ ]',''),'[.]',''),',','.').alias('VLLIQUIDO'),\
                                           df0._c6.cast("integer").alias('PEDIDO'), \
                                           df0._c7.cast("string").alias('FUNC'), \
                                           df0._c8.cast("integer").alias('CAIXA'), \
                                           df0._c9.cast("string").alias('UNIDADE'))\
                                           .withColumn('DATAINGESTAO', f.lit('20220620'))
print(f'Recebimento: {df1.count()}')

Recebimento: 8612


In [5]:
# df1.filter(df1.DESCRICAO == "FRANQUIA").show()

In [6]:
# df2 = df1.distinct()

In [7]:
# print(f'Contas a Pagar Formatados : {df2.count()}')
# df2.show()

In [8]:
df2 = df1.dropna(how='all', subset='MES_HIS')
print(f'Contas a Pagar : {df2.count()}')

Contas a Pagar : 8610


In [9]:
# df_contasapagar = df3.select(f.row_number().over(w).alias('ID'),'*')

In [10]:
# df_contasapagar.show(truncate=False)

In [11]:
df2.write.format("parquet").mode("overwrite").parquet('work/recebimentos')

In [12]:
# dfMelhorDia = df_contasapagar.groupBy("MES_PG","CATEGORIA").agg(f.format_number(f.sum("VALORPG"),2).alias("total"),\
#                              f.format_number(f.avg("VALORPG"),2).alias("media"),\
#                              f.format_number(f.min("VALORPG"),2).alias("menor"),\
#                              f.format_number(f.max("VALORPG"),2).alias("maior")).\
#     orderBy(["MES_PG","total"],ascending=[0, 0])

In [13]:
dfMelhorDia = df2.groupBy("MES_HIS","FORMAPG").agg(f.round(f.sum("VLLIQUIDO"),2).alias("total"),\
                             f.count("MES_HIS").alias("Compras"),\
                             f.round(f.avg("VLLIQUIDO"),2).alias("media"),\
                             f.round(f.min("VLLIQUIDO"),2).alias("menor"),\
                             f.round(f.max("VLLIQUIDO"),2).alias("maior")).\
    orderBy(["MES_HIS",'FORMAPG'],ascending=[1, 1])

In [14]:
dfMelhorDia.show(truncate=False,n=1000)


+-------+----------------------------------+--------+-------+-----+------+-----+
|MES_HIS|FORMAPG                           |total   |Compras|media|menor |maior|
+-------+----------------------------------+--------+-------+-----+------+-----+
|5      |Cartão de Crédito American Express|174.34  |6      |29.06|17.19 |51.59|
|5      |Cartão de Crédito Elo             |625.35  |31     |20.17|12.59 |9.69 |
|5      |Cartão de Crédito Mastercard      |15336.0 |645    |23.78|10.75 |9.77 |
|5      |Cartão de Crédito Outros          |75.07   |2      |37.54|36.07 |39.0 |
|5      |Cartão de Crédito Visa            |6734.2  |300    |22.45|10.26 |9.77 |
|5      |Cartão de Débito Elo              |5211.41 |221    |23.58|10.25 |93.24|
|5      |Cartão de Débito Mastercard       |28573.3 |1269   |22.52|10.32 |96.34|
|5      |Cartão de Débito Outros           |126.45  |7      |18.06|11.76 |9.8  |
|5      |Cartão de Débito Visa             |13092.79|582    |22.5 |10.81 |9.83 |
|5      |Cheque             